In [1]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

In [2]:
# Insert your Spotify username and the credentials that you obtained from spotify developer
cid = 'XXXXX'
secret = 'XXXXXXX'
redirect_uri='http://localhost:7777/callback'
username = 'XXXXXX'

In [3]:
# Once the Authorisation is complete, we just need to `sp` to call the APIs
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [4]:
# Getting top 50 tracks from user
results = sp.current_user_top_tracks(limit=50, offset=0,time_range='medium_term')

In [6]:
# Convert it to Dataframe
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_favourite = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_favourite.head()

,track_name,album,track_id,artist,duration,popularity
0,Prada Tune,Almost There,23slkWAmlcxi3XcSwC9RLZ,LUCKI,122813,45
1,Tarantino,Almost There,1CzjJzPOCLkZr2oqiOrXjc,LUCKI,114773,38
2,Find Me,Find Me,0CfHAgwAGHXS0bG8MvI5TV,Working On Dying,136146,39
3,Almost Back,Almost Back,176LvsOHbOXcgJgBBZ8wgs,LUCKI,91585,47
4,TUNE & SCOTTY,TUNE & SCOTTY,6xczQgrsVl3jFO3eEoNMK0,LUCKI,138120,27


In [7]:
# Getting features for each song
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']] 
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness'],
                              features['mode']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness','duration_ms', 'key',
                                                             'valence', 'speechiness', 'mode'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

In [8]:
df_fav = fetch_audio_features (sp, df_favourite)
df_fav.head()

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness,mode
track_name,,,,,,,,,,,,,
Prada Tune,23slkWAmlcxi3XcSwC9RLZ,0.706,0.02630,0.268,130.048,0.000000,-13.220,0.239,122813,5,0.330,0.2870,0
Tarantino,1CzjJzPOCLkZr2oqiOrXjc,0.790,0.04950,0.547,135.037,0.000000,-9.974,0.114,114773,4,0.318,0.1080,0
Find Me,0CfHAgwAGHXS0bG8MvI5TV,0.850,0.01840,0.540,120.968,0.000000,-8.434,0.293,136146,1,0.470,0.2140,1
Almost Back,176LvsOHbOXcgJgBBZ8wgs,0.764,0.00994,0.456,125.965,0.925000,-10.227,0.111,91585,2,0.227,0.2640,1
TUNE & SCOTTY,6xczQgrsVl3jFO3eEoNMK0,0.695,0.00137,0.672,133.007,0.000002,-10.004,0.126,138120,8,0.406,0.0469,1


In [10]:
# Getting featured playlists. (Hidden output)
response = sp.featured_playlists(limit = 50)
# check the format of the data 
response

{'message': "Editor's picks",
 'playlists': {'href': 'https://api.spotify.com/v1/browse/featured-playlists?timestamp=2021-03-15T17%3A09%3A34&offset=0&limit=50',
  'items': [{'collaborative': False,
    'description': 'Bruno Mars & Anderson .Paak are on top of the Hottest 50!',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DXcBWIGoYBM5M'},
    'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXcBWIGoYBM5M',
    'id': '37i9dQZF1DXcBWIGoYBM5M',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706f0000000335c3779dd12d3cdfc78eac80',
      'width': None}],
    'name': "Today's Top Hits",
    'owner': {'display_name': 'Spotify',
     'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
     'href': 'https://api.spotify.com/v1/users/spotify',
     'id': 'spotify',
     'type': 'user',
     'uri': 'spotify:user:spotify'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'MTYxNTc4Mjg5MiwwMDAwMDQ4ODAw